In [5]:
import gooseberry as gs 

import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib as mpl 
from matplotlib.patches import Ellipse
import numpy as np 

import jax 
import jax.numpy as jnp
from functools import partial 

from dynamics import QQT
from tools import BlindColours, zero_balanced_weights

In [70]:
"""
randomly initialised balanced weights of dimensions in_dim, 
hidden_dim, out_dim and balance coefficient c
""" 


def balanced_weights(in_dim, hidden_dim, out_dim, c):

    #random number bigger than c:
    a = np.random.exponential(scale = 5) + c 
    b = np.sqrt(a*a - c)

    #TODO: not sure which dimension, doesnt work for hidden_dim > min(out_dim, in_dim)
    S_1 = a * np.eye(max(hidden_dim, in_dim))[:hidden_dim, :in_dim]
    S_2 = b * np.eye(max(out_dim, hidden_dim))[:out_dim, :hidden_dim] 

    random_matrix_one = np.random.rand(hidden_dim, hidden_dim)
    random_matrix_two = np.random.rand(in_dim, in_dim)
    random_matrix_three = np.random.rand(out_dim, out_dim)

    q_one, _ = np.linalg.qr(random_matrix_one)
    q_two, _ = np.linalg.qr(random_matrix_two)
    q_three, _ = np.linalg.qr(random_matrix_three)

    U_1 = q_one 
    V_2 = q_one 
    V_1 = q_two
    U_2 = q_three

    W1 = U_1 @ S_1 @ (V_1.T)
    W2 = U_2 @ S_2 @ (V_2.T)

    return W1, W2 



In [60]:
def check_balance(w1, w2):
    return w1 @ w1.T - w2.T @ w2 

In [72]:
w1, w2 = balanced_weights(10, 2, 10, 5)

check_balance(w1, w2)

array([[ 5.00000000e+00, -1.58929512e-14],
       [-1.58929512e-14,  5.00000000e+00]])